# Massachusetts income tax

Massachusetts has a mostly flat income tax of five percent, with personal exemptions based on filing status and number of dependents.
It also has other exemptions, deductions, and credits, including an Earned Income Tax Credit set at 30 percent of a filer's federal EITC payment.

## Examples

Consider a single person.

In [11]:
from openfisca_us import IndividualSim
import pandas as pd
import plotly.express as px

sim = IndividualSim(year=2022)
sim.add_person(name="person")
sim.add_person(name="child")
sim.add_tax_unit(name="tax_unit", members=["person", "child"])
sim.add_household(name="household", members=["person", "child"], state_code="MA")

sim.vary("employment_income", max=100_000, step=1_000)

LABELS = dict(
    employment_income="Monthly employment income",
    income_source="Income source",
    state_income_tax="Massachusetts income tax",
    mtr="State marginal tax rate",
)

df = pd.DataFrame(
    dict(
        employment_income=sim.calc("employment_income")[0],
        state_income_tax=sim.calc("state_income_tax")[0],
        mtr=sim.deriv("state_income_tax", "employment_income", wrt_target="person"),
    )
)

fig = px.line(
    df,
    "employment_income",
    "state_income_tax",
    labels=LABELS,
    title="Massachusetts income tax for a one-person household",
)
fig.update_layout(xaxis_tickformat="$,", yaxis_tickformat="$,")
fig.show()

Their marginal tax rate goes from zero to five percent, depending on if their income exceeds their personal exemption.

In [5]:
fig = px.line(
    df,
    "employment_income",
    "mtr",
    labels=LABELS,
    title="Massachusetts marginal tax rate for a one-person household",
)
fig.update_layout(
    xaxis_tickformat="$,", yaxis_tickformat=".0%"#, yaxis_range=[0, 1]
)
fig.show()